In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm

In [2]:
# Reading Datasets
patent = pd.read_csv('PATENT.csv', index_col=False)
foreign = pd.read_csv('FOREIGN.csv', index_col=False)
employment = pd.read_csv('EMPLOYMENT.csv', index_col=False)
control = pd.read_csv('CONTROL.csv', index_col=False)
cumulative = pd.read_csv('CUMULATIVE.csv', index_col=False)
wage = pd.read_csv('WAGE.csv', index_col=False)

In [3]:
# Getting Cumulative Patent Counts
cols = cumulative.columns[1:]
cumulative[cols] = cumulative[cols].cumsum(axis=1)

# Drop pre-1990
cumulative = cumulative.drop('Pre-1990', axis=1)

In [4]:
# Summary Statistics
summary = wage.describe().T
formatted_summary = summary.style.format("{:.2f}")
display(formatted_summary)

,count,mean,std,min,25%,50%,75%,max
1990,14.00,10.80,1.78,8.53,9.22,10.77,11.69,14.44
1991,14.00,11.15,1.89,8.75,9.51,11.16,12.06,15.12
1992,14.00,11.43,1.95,9.01,9.78,11.37,12.34,15.59
1993,14.00,11.71,2.04,9.25,10.01,11.62,12.64,16.21
1994,14.00,12.00,2.15,9.52,10.17,11.88,12.88,16.93
1995,14.00,12.27,2.18,9.76,10.41,12.10,13.10,17.21
1996,14.00,12.66,2.22,10.09,10.73,12.50,13.47,17.66
1997,14.00,13.02,2.23,10.50,11.04,12.91,13.90,17.99
1998,14.00,13.35,2.18,10.85,11.33,13.28,14.22,17.91
1999,14.00,13.73,2.16,11.18,11.73,13.66,14.73,18.24


In [5]:
# Test Data Frame
print(wage.head())

     NAICS    1990    1991    1992   1993   1994   1995    1996   1997   1998  \
0      311   9.040   9.320   9.590   9.82  10.00  10.26  10.500  10.77  11.09   
1      321   8.820   9.020   9.240   9.40   9.66   9.92  10.240  10.53  10.85   
2  322,323  11.585  11.885  12.155  12.40  12.69  13.01  13.395  13.77  14.20   
3      325  12.850  13.300  13.700  13.97  14.33  14.86  15.370  15.78  16.23   
4      326   9.760  10.070  10.350  10.56  10.67  10.86  11.170  11.48  11.79   

   ...   2001   2002   2003   2004    2005    2006    2007   2008   2009  \
0  ...  12.18  12.55  12.80  12.98  13.040  13.130  13.550  14.00  14.39   
1  ...  11.99  12.33  12.71  13.03  13.160  13.390  13.680  14.19  14.92   
2  ...  15.43  15.89  16.35  16.81  16.865  16.905  17.295  17.82  18.02   
3  ...  17.57  17.97  18.50  19.17  19.670  19.600  19.550  19.50  20.30   
4  ...  13.21  13.55  14.18  14.59  14.800  14.970  15.390  15.85  16.01   

     2010  
0  14.410  
1  14.850  
2  18.475  
3  21.07

In [6]:
# #Summary for Employment Variable
# employment.set_index('NAICS', inplace=True)
# employment_T = employment.T

# # Plot the data
# ax = employment_T.plot(figsize=(8, 5), linewidth=2, fontsize=12)

# # Set the axis labels and title
# ax.set_xlabel('Year', fontsize=12)
# ax.set_ylabel('Employed Workers (in thousands)', fontsize=12)
# ax.set_title('Employment by NAICS code over time', fontsize=14)

# # Show the plot
# plt.show()

In [7]:
# #Summary for Patent Variable
# patent.set_index('NAICS', inplace=True)
# patent_T = patent.T

# # Plot the data
# ax = patent_T.plot(figsize=(8, 5), linewidth=2, fontsize=12)

# # Set the axis labels and title
# ax.set_xlabel('Year', fontsize=12)
# ax.set_ylabel('Annual Registered Patents', fontsize=12)
# ax.set_title('Patents by NAICS code over time', fontsize=14)

# # Show the plot
# plt.show()

In [8]:
# #Summary for Cumulative Patent Variable
# cumulative.set_index('NAICS', inplace=True)
# cumulative_T = cumulative.T

# # Plot the data
# ax = cumulative_T.plot(figsize=(8, 5), linewidth=2, fontsize=12)

# # Set the axis labels and title
# ax.set_xlabel('Year', fontsize=12)
# ax.set_ylabel('Cumulative Registered Patents', fontsize=12)
# ax.set_title('Cumulative Patent Counts by NAICS code over time', fontsize=14)

# # Show the plot
# plt.show()

In [9]:
# #Summary for Wage Variable
# wage.set_index('NAICS', inplace=True)
# wage_T = wage.T

# # Plot the data
# ax = wage_T.plot(figsize=(8, 5), linewidth=2, fontsize=12)

# # Set the axis labels and title
# ax.set_xlabel('Year', fontsize=12)
# ax.set_ylabel('Average Wage (in USD)', fontsize=12)
# ax.set_title('Average Wage by NAICS code over time', fontsize=14)

# # Show the plot
# plt.show()

In [10]:
# Merge Employment and Patent Data Frames
melted_employment = employment.reset_index(drop=True).melt(id_vars='NAICS', var_name='Year', value_name='employment')
melted_patent = patent.reset_index(drop=True).melt(id_vars='NAICS', var_name='Year', value_name='patent')
# Add Patent t-1 column to Data Frame
melted_patent['patent_t-1'] = melted_patent.groupby('NAICS')['patent'].shift(1)
# Add Patent t-2 column to Data Frame
melted_patent['patent_t-2'] = melted_patent.groupby('NAICS')['patent'].shift(2)
complete_df = pd.merge(melted_employment, melted_patent, on=['NAICS', 'Year'])

In [11]:
# Test Merged Data Frame
print(complete_df.head())

     NAICS  Year   employment  patent  patent_t-1  patent_t-2
0      311  1990  1506.741667     368       368.0       258.0
1      321  1990   542.991667     173       201.0       151.0
2  322,323  1990  1455.633333     507       477.0       380.0
3      325  1990  1035.833333    6275      6554.0      5275.0
4      326  1990   824.900000    2537      2624.0      2126.0


In [12]:
# Add Cumulative Patent Data Frame
melted_cumulative = cumulative.reset_index(drop=True).melt(id_vars='NAICS', var_name='Year', value_name='cum_patent')
complete_df = pd.merge(complete_df, melted_cumulative, on=['NAICS', 'Year'])

In [13]:
# Test Merged Data Frame
print(complete_df.head())

     NAICS  Year   employment  patent  patent_t-1  patent_t-2  cum_patent
0      311  1990  1506.741667     368       368.0       258.0        7688
1      321  1990   542.991667     173       201.0       151.0        4629
2  322,323  1990  1455.633333     507       477.0       380.0       10348
3      325  1990  1035.833333    6275      6554.0      5275.0      168435
4      326  1990   824.900000    2537      2624.0      2126.0       60552


In [14]:
# Add Wage Data Frame
melted_wage = wage.reset_index(drop=True).melt(id_vars='NAICS', var_name='Year', value_name='wage')
complete_df = pd.merge(complete_df, melted_wage, on=['NAICS', 'Year'])

In [15]:
# Test Merged Data Frame
print(complete_df.head())

     NAICS  Year   employment  patent  patent_t-1  patent_t-2  cum_patent  \
0      311  1990  1506.741667     368       368.0       258.0        7688   
1      321  1990   542.991667     173       201.0       151.0        4629   
2  322,323  1990  1455.633333     507       477.0       380.0       10348   
3      325  1990  1035.833333    6275      6554.0      5275.0      168435   
4      326  1990   824.900000    2537      2624.0      2126.0       60552   

     wage  
0   9.040  
1   8.820  
2  11.585  
3  12.850  
4   9.760  


In [16]:
# Merge with Control Variables
control.rename(columns={'year': 'Year'}, inplace=True)
complete_df['Year'] = complete_df['Year'].astype('int64')
complete_df = pd.merge(complete_df, control.reset_index(drop=True), on='Year')

In [17]:
# Test Merged Data Frame
print(complete_df.head())
print(complete_df.shape[0])

     NAICS  Year   employment  patent  patent_t-1  patent_t-2  cum_patent  \
0      311  1990  1506.741667     368       368.0       258.0        7688   
1      321  1990   542.991667     173       201.0       151.0        4629   
2  322,323  1990  1455.633333     507       477.0       380.0       10348   
3      325  1990  1035.833333    6275      6554.0      5275.0      168435   
4      326  1990   824.900000    2537      2624.0      2126.0       60552   

     wage  gdp_rate       cpi      labor        tax    fdi_in   fdi_out  \
0   9.040  1.885965  59.91976  127352635  30.553273  1.194504  1.005174   
1   8.820  1.885965  59.91976  127352635  30.553273  1.194504  1.005174   
2  11.585  1.885965  59.91976  127352635  30.553273  1.194504  1.005174   
3  12.850  1.885965  59.91976  127352635  30.553273  1.194504  1.005174   
4   9.760  1.885965  59.91976  127352635  30.553273  1.194504  1.005174   

   unem_spend  min_wage      debt    r&d  
0       0.417       3.8  53.62628  2.556  


In [18]:
#Summary Statistics
summary = complete_df.describe().T
formatted_summary = summary.style.format("{:.2f}")
display(formatted_summary)

,count,mean,std,min,25%,50%,75%,max
Year,273.00,2000.00,6.07,1990.00,1995.00,2000.00,2005.00,2010.00
employment,273.00,1041.11,491.23,341.93,576.39,950.86,1480.30,2134.88
patent,273.00,5070.07,8541.48,105.00,349.00,2414.00,5658.00,57038.00
patent_t-1,273.00,4876.41,7940.91,105.00,350.00,2368.00,5649.00,45498.00
patent_t-2,273.00,4732.67,7570.37,105.00,365.00,2337.00,5573.00,45498.00
cum_patent,273.00,131600.91,160542.75,4629.00,13264.00,71985.00,190835.00,852019.00
wage,273.00,14.51,3.29,8.53,12.12,14.19,16.64,25.23
gdp_rate,273.00,2.58,1.77,-2.60,1.89,2.78,3.85,4.79
cpi,273.00,79.78,12.31,59.92,69.88,78.97,89.56,100.00
labor,273.00,144143575.19,9771837.40,127352635.00,135975699.00,146165420.00,152044687.00,157377616.00


In [19]:
# Regression on patent without Control Variables

# Define the X and Y variables
X = complete_df[['patent']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_1 = sm.OLS(Y, X).fit()

# Print the model summary
print(model_1.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     24.10
Date:                Thu, 11 May 2023   Prob (F-statistic):           1.58e-06
Time:                        14:52:24   Log-Likelihood:                -2067.0
No. Observations:                 273   AIC:                             4138.
Df Residuals:                     271   BIC:                             4145.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        957.7879     33.209     28.841      0.0

In [20]:
# Regression on cumulative patent without Control Variables

# Define the X and Y variables
X = complete_df[['cum_patent']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_2 = sm.OLS(Y, X).fit()

# Print the model summary
print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.092
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     27.32
Date:                Thu, 11 May 2023   Prob (F-statistic):           3.44e-07
Time:                        14:52:24   Log-Likelihood:                -2065.5
No. Observations:                 273   AIC:                             4135.
Df Residuals:                     271   BIC:                             4142.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        919.2487     36.735     25.024      0.0

In [21]:
# Regression

# Define the X and Y variables
X = complete_df[['patent', 'gdp_rate', 'cpi', 'labor', 'tax', 'fdi_in', 'fdi_out', 'unem_spend', 'min_wage', 'debt']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_3 = sm.OLS(Y, X).fit()

# Print the model summary
print(model_3.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.111
Method:                 Least Squares   F-statistic:                     4.404
Date:                Thu, 11 May 2023   Prob (F-statistic):           9.91e-06
Time:                        14:52:24   Log-Likelihood:                -2057.4
No. Observations:                 273   AIC:                             4137.
Df Residuals:                     262   BIC:                             4177.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -134.6559   6222.328     -0.022      0.9

In [22]:
# Regression with Cumulative Patent Count

# Define the X and Y variables
X = complete_df[['cum_patent', 'gdp_rate', 'cpi', 'labor', 'tax', 'fdi_in', 'fdi_out', 'unem_spend', 'min_wage', 'debt']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_4 = sm.OLS(Y, X).fit()

# Print the model summary
print(model_4.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.171
Model:                            OLS   Adj. R-squared:                  0.139
Method:                 Least Squares   F-statistic:                     5.393
Date:                Thu, 11 May 2023   Prob (F-statistic):           2.93e-07
Time:                        14:52:24   Log-Likelihood:                -2053.1
No. Observations:                 273   AIC:                             4128.
Df Residuals:                     262   BIC:                             4168.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -782.1813   6122.618     -0.128      0.8

In [23]:
# Regression with Patent Count and Wage

# Define the X and Y variables
X = complete_df[['patent', 'wage', 'gdp_rate', 'cpi', 'labor', 'tax', 'fdi_in', 'fdi_out', 'unem_spend', 'debt']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_5 = sm.OLS(Y, X).fit()

# Print the model summary
print(model_5.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.225
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     7.621
Date:                Thu, 11 May 2023   Prob (F-statistic):           1.13e-10
Time:                        14:52:24   Log-Likelihood:                -2043.8
No. Observations:                 273   AIC:                             4110.
Df Residuals:                     262   BIC:                             4149.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -222.6856   5903.504     -0.038      0.9

In [24]:
# Regression with Patent Count and Wage

# Define the X and Y variables
X = complete_df[['cum_patent', 'wage', 'gdp_rate', 'cpi', 'labor', 'tax', 'fdi_in', 'fdi_out', 'unem_spend', 'debt']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_5b = sm.OLS(Y, X).fit()

# Print the model summary
print(model_5b.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     8.206
Date:                Thu, 11 May 2023   Prob (F-statistic):           1.49e-11
Time:                        14:52:25   Log-Likelihood:                -2041.4
No. Observations:                 273   AIC:                             4105.
Df Residuals:                     262   BIC:                             4145.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -685.4858   5851.857     -0.117      0.9

In [25]:
# Regression with Patent Count and Wage

# Define the X and Y variables
X = complete_df[['patent', 'patent_t-1', 'patent_t-2', 'gdp_rate', 'cpi', 'labor', 'tax', 'fdi_in', 'fdi_out', 'unem_spend','min_wage', 'debt']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_6 = sm.OLS(Y, X).fit()

# Print the model summary
print(model_6.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.146
Model:                            OLS   Adj. R-squared:                  0.107
Method:                 Least Squares   F-statistic:                     3.703
Date:                Thu, 11 May 2023   Prob (F-statistic):           3.61e-05
Time:                        14:52:25   Log-Likelihood:                -2057.1
No. Observations:                 273   AIC:                             4140.
Df Residuals:                     260   BIC:                             4187.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -521.6853   6333.265     -0.082      0.9

In [26]:
# Regression with Patent Count and Wage

# Define the X and Y variables
X = complete_df[['patent', 'r&d', 'gdp_rate', 'cpi', 'labor', 'tax', 'fdi_in', 'fdi_out', 'unem_spend', 'min_wage', 'debt']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_7 = sm.OLS(Y, X).fit()

# Print the model summary
print(model_7.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.144
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     3.988
Date:                Thu, 11 May 2023   Prob (F-statistic):           2.19e-05
Time:                        14:52:25   Log-Likelihood:                -2057.4
No. Observations:                 273   AIC:                             4139.
Df Residuals:                     261   BIC:                             4182.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -18.9049   7524.529     -0.003      0.9

In [27]:
# Regression with Patent Count and Wage

# Define the X and Y variables
X = complete_df[['cum_patent', 'r&d', 'wage', 'gdp_rate', 'cpi', 'labor', 'tax', 'fdi_in', 'fdi_out', 'unem_spend', 'debt']]
Y = complete_df['employment']

# Add a constant term to the X variable
X = sm.add_constant(X)

# Fit the OLS regression model
model_8 = sm.OLS(Y, X).fit()

# Print the model summary
print(model_8.summary())

                            OLS Regression Results                            
Dep. Variable:             employment   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.207
Method:                 Least Squares   F-statistic:                     7.435
Date:                Thu, 11 May 2023   Prob (F-statistic):           4.38e-11
Time:                        14:52:25   Log-Likelihood:                -2041.4
No. Observations:                 273   AIC:                             4107.
Df Residuals:                     261   BIC:                             4150.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -1071.6447   6288.366     -0.170      0.8

In [28]:
# Create a dictionary to store the summary statistics for each model
summary_dict = {
    'Model 1': model_1.summary2(),
    'Model 2': model_2.summary2(),
    'Model 3': model_3.summary2(),
    'Model 4': model_4.summary2()
}

# Create a DataFrame to store the summary statistics for all models
summary_table = pd.DataFrame(columns=['Model', 'Summary'])

# Populate the DataFrame with model names and summary statistics
for model_name, summary in summary_dict.items():
    summary_table = summary_table.append({'Model': model_name, 'Summary': summary.tables[1]}, ignore_index=True)

# Print or display the summary table
for _, row in summary_table.iterrows():
    print(row['Model'])
    print(row['Summary'])
    print('\n' + '=' * 50 + '\n')  # Print separator between models

Model 1
             Coef.   Std.Err.          t         P>|t|      [0.025  \
const   957.787887  33.209035  28.841184  1.425817e-84  892.407390   
patent    0.016434   0.003348   4.908901  1.583979e-06    0.009843   

             0.975]  
const   1023.168385  
patent     0.023026  


Model 2
                 Coef.   Std.Err.          t         P>|t|      [0.025  \
const       919.248712  36.734754  25.023951  2.057891e-72  846.926932   
cum_patent    0.000926   0.000177   5.227098  3.443113e-07    0.000577   

                0.975]  
const       991.570492  
cum_patent    0.001275  


Model 3
                 Coef.     Std.Err.         t         P>|t|        [0.025  \
const      -134.655934  6222.327780 -0.021641  9.827510e-01 -12386.790900   
patent        0.017804     0.003317  5.367803  1.756018e-07      0.011273   
gdp_rate     -0.271658    28.015550 -0.009697  9.922707e-01    -55.435948   
cpi         -18.607892    25.014384 -0.743888  4.576110e-01    -67.862709   
labor       

In [29]:
reg_summaries = [model_1.summary(), model_2.summary(), model_3.summary()]

# create a list of dictionaries to store the relevant information for each model
model_dicts = []

for summary in reg_summaries:
    # get the coefficient and standard error of each variable
    coef = summary.params
    std_err = summary.bse
    
    # get the observation count and R-squared value
    nobs = summary.nobs
    rsquared = summary.rsquared
    
    # create a dictionary with the relevant information
    model_dict = {
        'coef': coef,
        'std_err': std_err,
        'nobs': nobs,
        'rsquared': rsquared
    }
    
    # add the dictionary to the list of model dictionaries
    model_dicts.append(model_dict)

# create a list of variable names
var_names = list(reg_summaries[0].params.index)

# create an empty dataframe with columns for the relevant information
results_df = pd.DataFrame(columns=['Variable', 'Model 1 Coef', 'Model 1 SE', 'Model 2 Coef', 'Model 2 SE', 'Model 3 Coef', 'Model 3 SE', 'Model 1 nobs', 'Model 1 R-squared', 'Model 2 nobs', 'Model 2 R-squared', 'Model 3 nobs', 'Model 3 R-squared'])

# loop through the variable names and add the information for each variable to the dataframe
for var_name in var_names:
    row_dict = {'Variable': var_name}
    for i, model_dict in enumerate(model_dicts):
        row_dict[f'Model {i+1} Coef'] = model_dict['coef'][var_name]
        row_dict[f'Model {i+1} SE'] = model_dict['std_err'][var_name]
        row_dict[f'Model {i+1} nobs'] = model_dict['nobs']
        row_dict[f'Model {i+1} R-squared'] = model_dict['rsquared']
    results_df = results_df.append(row_dict, ignore_index=True)

# print the dataframe
print(results_df)

AttributeError: 'Summary' object has no attribute 'params'